In [1]:
import pandas as pd
import numpy as np

Files in the folder

In [4]:
lst = set()
formulation_set = set()
avoided_formulation = ["inject", "im-iv", "intravenous", "intra-articular", "intramuscular"]

def isAcceptedDrugFormulation (formulation):
    for avoided_form in avoided_formulation:
        if formulation.find(avoided_form) >= 0:
            return False
    return True

with open ("Products.txt", "r") as f:
    f.readline()
    c = 0
    for line in f:
        line_lst = line.split("\t")
        formulation = line_lst[2].lower()
        drugname = line_lst[5].lower()
        if isAcceptedDrugFormulation(formulation):
            lst.add(drugname)
            formulation_set.add(formulation)
        c += 1
lst = list(lst)
print("total lines in the file: ", c)
print("Sample of drug names: ", list(lst)[:10])
print("number of drugs: ", len(lst))

total lines in the file:  44873
Sample of drug names:  ['prolensa', 'ssd', 'fluocinonide', 'cardizem cd', 'scemblix', 'carglumic acid', 'emsam', 'horizant', 'fexofenadine hydrochloride hives', 'synovalyte in plastic container']
number of drugs:  5583


Generate numpy array of drugs

Provide list of drug names in numpy in case my teammate needs. 

In [5]:
drugset_np = np.array(list(lst))
drugset_np.shape

(5583,)

In [6]:
print(len(lst))

5583


Feature generation step
four types of error
1. omission (missing one letter)
2. transpose (order of two neighbor letters are incorrect)
3. replacement
a. vow-to-vow replacement
b. consonant-to-consonant replacement
4. insertion (insert random letter)



In [7]:


def omission(drug, i):
    if i == 0:
        return drug[i+1:]
    elif i == len(drug)-1:
        return drug[:i]
    else:
        return drug[:i] + drug[i+1:]

In [8]:
drug = "abcdef"
for i in range(len(drug)):
    print(omission (drug, i))

bcdef
acdef
abdef
abcef
abcdf
abcde


In [9]:
def transpose(drug, i):
    if i >= len(drug)-1-1:
        i = len(drug)-2
        return drug[:i] + drug[i+1] +drug[i]
    else:
        return drug[:i] + drug[i+1] +drug[i] + drug[i+2:]
        


In [10]:
drug = "abcdef"
for i in range(len(drug)):
    print(i)
    print(transpose(drug, i))

0
bacdef
1
acbdef
2
abdcef
3
abcedf
4
abcdfe
5
abcdfe


In [11]:
import random
VOWELS = list("aeiou")
CONSONANTS = list("bcdfghjklmnpqrstvwxyz")
ALL_LETTERS = VOWELS + CONSONANTS

print(VOWELS)
print(CONSONANTS)
print(ALL_LETTERS)
def replace(drug, i):
    c = drug[i].lower()
    if c in VOWELS:
        r = random.choice(VOWELS)
    elif c in CONSONANTS:
        r = random.choice(CONSONANTS)
    else:
        r = random.choice(ALL_LETTERS)
    ans = drug[:i] + r + drug[i+1:]
    if ans == drug:
        replace(drug, i)
    return ans

['a', 'e', 'i', 'o', 'u']
['b', 'c', 'd', 'f', 'g', 'h', 'j', 'k', 'l', 'm', 'n', 'p', 'q', 'r', 's', 't', 'v', 'w', 'x', 'y', 'z']
['a', 'e', 'i', 'o', 'u', 'b', 'c', 'd', 'f', 'g', 'h', 'j', 'k', 'l', 'm', 'n', 'p', 'q', 'r', 's', 't', 'v', 'w', 'x', 'y', 'z']


In [12]:
drug = "abc__def"
for i in range(len(drug)):
    print(i)
    print(replace(drug, i))

0
ubc__def
1
abc__def
2
abl__def
3
abcd_def
4
abc_xdef
5
abc__qef
6
abc__dif
7
abc__den


In [13]:
import random
ALL_LETTERS = VOWELS + CONSONANTS

print(ALL_LETTERS)

def insertion(drug, i):
    r = random.choice(ALL_LETTERS)
    return drug[:i] + r + drug[i:]

['a', 'e', 'i', 'o', 'u', 'b', 'c', 'd', 'f', 'g', 'h', 'j', 'k', 'l', 'm', 'n', 'p', 'q', 'r', 's', 't', 'v', 'w', 'x', 'y', 'z']


In [14]:
drug = "abcdef"
for i in range(len(drug)+1):
    print(insertion(drug, i))

yabcdef
akbcdef
abwcdef
abcfdef
abcdnef
abcdekf
abcdefq


In [15]:
# this feature can be used for unsupervised learning and transformer data intake
def genUnsprvFeatPair(drug):
    temp = [omission(drug, i) for i in range(len(drug))]
    temp += [transpose(drug, i) for i in range(len(drug)-1)]
    temp += [replace(drug, i) for i in range(len(drug))]
    temp += [insertion(drug, i) for i in range(len(drug))]
    return [(x, drug) for x in temp]
    

In [16]:
drug = "drug name"
print(genUnsprvFeatPair(drug))

[('rug name', 'drug name'), ('dug name', 'drug name'), ('drg name', 'drug name'), ('dru name', 'drug name'), ('drugname', 'drug name'), ('drug ame', 'drug name'), ('drug nme', 'drug name'), ('drug nae', 'drug name'), ('drug nam', 'drug name'), ('rdug name', 'drug name'), ('durg name', 'drug name'), ('drgu name', 'drug name'), ('dru gname', 'drug name'), ('drugn ame', 'drug name'), ('drug anme', 'drug name'), ('drug nmae', 'drug name'), ('drug naem', 'drug name'), ('prug name', 'drug name'), ('dpug name', 'drug name'), ('dreg name', 'drug name'), ('drup name', 'drug name'), ('druganame', 'drug name'), ('drug yame', 'drug name'), ('drug nome', 'drug name'), ('drug nage', 'drug name'), ('drug namu', 'drug name'), ('idrug name', 'drug name'), ('dhrug name', 'drug name'), ('drpug name', 'drug name'), ('drujg name', 'drug name'), ('drugf name', 'drug name'), ('drug uname', 'drug name'), ('drug nuame', 'drug name'), ('drug nafme', 'drug name'), ('drug namge', 'drug name')]


reference: dictionary spreader: https://sparrow.dev/object-spread-operator-python/

In [17]:
# this feature can be used for NLTK ML models (naive bayes, sklearn models)
def genSprvFeatPair(drug):
    temp = {f"omission_{i}":omission(drug, i) for i in range(len(drug))}
    temp2 = {f"transpose_{i}": transpose(drug, i) for i in range(len(drug)-1)}
    temp3 = {f"replace_{i}": replace(drug, i) for i in range(len(drug))}
    temp4 = {f"insertion_{i}":insertion(drug, i) for i in range(len(drug))}
    return [{**temp, **temp2, **temp3, **temp4}, drug]


In [18]:
#test
drug = "drug"
print(genSprvFeatPair(drug))

[{'omission_0': 'rug', 'omission_1': 'dug', 'omission_2': 'drg', 'omission_3': 'dru', 'transpose_0': 'rdug', 'transpose_1': 'durg', 'transpose_2': 'drgu', 'replace_0': 'trug', 'replace_1': 'dkug', 'replace_2': 'drug', 'replace_3': 'druj', 'insertion_0': 'bdrug', 'insertion_1': 'dsrug', 'insertion_2': 'drrug', 'insertion_3': 'drucg'}, 'drug']


In [19]:
len(lst)

5583

In [20]:
# unsprv_dataset only first 100. convert the whole dataset would cause too much time? 
Unsprv_dataset = sum([genUnsprvFeatPair(drug) for drug in lst[:]], [])
print(Unsprv_dataset[:10])
print(len(Unsprv_dataset))

[('rolensa', 'prolensa'), ('polensa', 'prolensa'), ('prlensa', 'prolensa'), ('proensa', 'prolensa'), ('prolnsa', 'prolensa'), ('prolesa', 'prolensa'), ('prolena', 'prolensa'), ('prolens', 'prolensa'), ('rpolensa', 'prolensa'), ('porlensa', 'prolensa')]
294741


In [21]:
supervised_dataset = [genSprvFeatPair(drug) for drug in lst[:1000]]
print(supervised_dataset[:2])
print(len(supervised_dataset))

[[{'omission_0': 'rolensa', 'omission_1': 'polensa', 'omission_2': 'prlensa', 'omission_3': 'proensa', 'omission_4': 'prolnsa', 'omission_5': 'prolesa', 'omission_6': 'prolena', 'omission_7': 'prolens', 'transpose_0': 'rpolensa', 'transpose_1': 'porlensa', 'transpose_2': 'prloensa', 'transpose_3': 'proelnsa', 'transpose_4': 'prolnesa', 'transpose_5': 'prolesna', 'transpose_6': 'prolenas', 'replace_0': 'mrolensa', 'replace_1': 'pwolensa', 'replace_2': 'prelensa', 'replace_3': 'proxensa', 'replace_4': 'prolansa', 'replace_5': 'proletsa', 'replace_6': 'prolenka', 'replace_7': 'prolensi', 'insertion_0': 'kprolensa', 'insertion_1': 'pfrolensa', 'insertion_2': 'prsolensa', 'insertion_3': 'proplensa', 'insertion_4': 'prolhensa', 'insertion_5': 'prolelnsa', 'insertion_6': 'prolenlsa', 'insertion_7': 'prolensja'}, 'prolensa'], [{'omission_0': 'sd', 'omission_1': 'sd', 'omission_2': 'ss', 'transpose_0': 'ssd', 'transpose_1': 'sds', 'replace_0': 'fsd', 'replace_1': 'shd', 'replace_2': 'ssl', 'ins

In [22]:
import nltk

Linear regression

In [24]:
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC

In [25]:
split = 0.80
size = int(len(supervised_dataset)*split)
print("size: ", size)
import random

random.shuffle(supervised_dataset)
print("length of features: ", len(supervised_dataset))
train_set, test_set = supervised_dataset[:size], supervised_dataset[size:]

size:  800
length of features:  1000


In [28]:
LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(train_set)
print("LinearSVC_classifier accuracy:", (nltk.classify.accuracy(LinearSVC_classifier, test_set))*100)

LinearSVC_classifier accuracy percent: 0.0


Logistic regression

In [27]:

LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(train_set)
print("LogisticRegression_classifier accuracy:", (nltk.classify.accuracy(LogisticRegression_classifier, test_set)))

LogisticRegression_classifier accuracy percent: 0.0


refrences:https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/